In [5]:
import torch
import pytorch_lightning as pl
import importlib
import numpy as np
import wandb
# from triplet_vade import TripletVaDE
import triplet_vade
importlib.reload(triplet_vade)
from autoencoder import SimpleAutoencoder, VaDE, ClusteringEvaluationCallback, cluster_acc

In [2]:
pretriained_model = 'pretrained_models/radiant-surf-28/autoencoder-epoch=55-loss=0.011.ckpt'
hparams = {'lr': 2e-3, 'lr_gmm': 2e-3, 'batch_size': 2**6, 'pretrain_epochs': 50, 'triplet_loss_margin': 0.5, 'triplet_loss_alpha': 0.0, 'warmup_epochs':10, 'triplet_loss_alpha_kl': 1., 'n_samples_for_triplets': None, 'data_size': None, 'pretrained_model_file': pretriained_model}
for i in range(3):
    triplets_model = triplet_vade.TripletVaDE(n_neurons=[784, 512, 512, 2048, 10], **hparams)
    # wandb.init()
    logger =  pl.loggers.wandb.WandbLogger(project='VaDE Triplets', group='triplet_loss_limited_data')

    callbacks = [ClusteringEvaluationCallback(on_start=True), 
                 ClusteringEvaluationCallback(on_start=True, ds_type='valid'),
                 pl.callbacks.LearningRateMonitor()]
    logger.log_hyperparams(hparams)
    trainer = pl.Trainer(gpus=1, callbacks=callbacks, logger=logger, 
                         log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=50)
    trainer.fit(triplets_model)
    wandb.finish()
    # triplets_model.save_hyperparameters()

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 1.7 M 
1 | decoder | Sequential | 1.7 M 
---------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | VaDE | 3.5 M 
-------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

TypeError: on_train_epoch_end() takes 3 positional arguments but 4 were given

In [ ]:
hparams = {'lr': 2e-3, 'lr_gmm': 2e-3, 'batch_size': 2**8, 'pretrain_epochs': 50, 'triplet_loss_margin': 0.5, 'triplet_loss_alpha': 00.0, 'warmup_epochs':10, 'triplet_loss_alpha_kl': 1., 'n_samples_for_triplets': 1000}
for i in range(10):
    triplets_model = triplet_vade.TripletVaDE(n_neurons=[784, 512, 512, 2048, 10], **hparams)
    # wandb.init()
    logger =  pl.loggers.wandb.WandbLogger(project='VaDE Triplets', group='triplet_loss_limited_data')

    callbacks = [ClusteringEvaluationCallback(on_start=True), 
                 ClusteringEvaluationCallback(on_start=True, ds_type='valid'),
                 pl.callbacks.LearningRateMonitor()]
    logger.log_hyperparams(hparams)
    trainer = pl.Trainer(gpus=1, callbacks=callbacks, logger=logger, 
                         log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=100)
    trainer.fit(triplets_model)
    wandb.finish()

In [19]:
model.load_state_dict(torch.load('trained_models/AE1.h5'))

<All keys matched successfully>

In [8]:
from sklearn import metrics

model.cuda()
true_y, predicted_y, X_encoded = model.cluster_data(method='kmeans')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-full')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-diag')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

0.7538434361452869
0.8249112629202103
0.7238449211641871


In [10]:
from sklearn.mixture import GaussianMixture
scores = []
for i in range(5):
    gmm = GaussianMixture(n_components=10, covariance_type='full', n_init=5)
    predicted_y = gmm.fit_predict(X_encoded)
    scores.append(metrics.normalized_mutual_info_score(true_y, predicted_y))
    
print(scores)
print(np.mean(scores), np.max(scores), np.std(scores))

[0.8206645765011308, 0.8503932657020613, 0.7995011494626447, 0.8239846028460401, 0.8483506486929793]
0.8285788486409713 0.8503932657020613 0.018952826999921197
